<a href="https://colab.research.google.com/github/sus4157d/almabetter/blob/main/Online_retail_customer_segementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - Online Retail Customer Segmentation



##### **Project Type**    - Unsupervised

##### **Contribution**    - Individual

# **Project Summary -**

We have been given  all transational data from 1-Dec-2010 to 9-Dec-2011 of a Uk based non-store online retail company . Customers of company are mostly wholesellers. We are required to identify the major customer segments. We started importing the library and loading CSV file in DataFrame df. We explored the how Head and Tail of df look like and checked the shape. we have 541909 row of datapoints with 8 columns. Out of Eight columns we have 'InvoiceNo','Stockcode','Description' as first three . Dtype of these is 'object', 'InvoiceNo' is unique transation No which have 'c' latter added if order was cancelled. we will drop all orders which were cancelled later on. Next is 'Stockcode' which is a unique code for each individual item used to identify in store. 'Description' is what is this item which was brought. 'InvoiceDate' gives Date and Time of transaction . Next are 'Quantity' which is int and tells how many items are brought . 'UnitPrice' is float tells what is price per item brought. 'CustomerID' is int again and its unique Id of each customer as we know. Last but not least we have 'Country' that tells us country name of customer . Description Check gives Min , Mean, Max values of "Quantity" , "UnitPrice" and "CustomerID" , we can say we have anomalies in first two as there is huge difference between 99percent and max value. We will take care of anomalies later on. We have 135 thousand missing value in CustomerID we cannot predict anything from such transation . So we need to drop these transation. From Duplicate value of 5 thousand we droped all keeping first. we droped all transation of invoice with 'c' which were cancelled.

Now we will start with Ploting Distribution of Feature Variable , EDA , Data visualization and Data Wrangling . Fistly we have ploted Distribution of "Quantity" and "UnitPrice" , Both gives Highly Right Skewed Plot . We have applied some Transformation and after that it became normal. We created new Feature from invoice Date like mounth, time, day, day_name, month_name . From feature Quantity and UnitPrice we created TotalAmount of each Transation.Created RFM model called Recency , Frequency and Monetary Model which model the rating of recency of each customer , frequency of each customer and How much amount each customer spend. From these rating we can rate each customer according to value of R, F, M Feature. Next is Data visualization , we ploted top five Product according to sales . Top Countries according to sales and Top Stock also .
We have ploted graph of Day_of_week, Hour_of_day and Month_of_year with respect to sales . We did Hypothesis testing , Do Recent Customer tends to spend more , Do Frequent Customer spend More or Less. Now lets start Featue Engineering like droping anomalies, Log Transformation and Scaling using StanderdScaler. Let start with Model Implementation , we have fitted K-Means, DBSCAN, and HIERICIAL Clustering technique with Silhouette score, elbow method and dendogram to find Optimum no of clusters.I finished with number two for optimum clusters using differnt method. lets begin the project.

# **GitHub Link -**

https://github.com/sus4157d/almabetter.git

# **Problem Statement**


**In this Project We are required to Identify Major Customer Segments of the data set. we are given dataset of all the transation of Uk based non-store ONLINE RETAIL. This company mainly sells all occassion gifts and mostly the customer are whole seller.**

# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Importing the libraries
import numpy as np
import pandas as pd
from numpy import math
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.ensemble import IsolationForest

from scipy.stats import *
import statistics

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
from seaborn.matrix import dendrogram

import warnings
from pylab import rcParams
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
warnings.filterwarnings('ignore')

### Dataset Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load Dataset
dfn=pd.read_excel("/content/drive/MyDrive/Online_Retail.xlsx",parse_dates=['InvoiceDate'])

In [ ]:
#Creating a copy of Dataset
df=dfn.copy()

### Dataset First View

In [ ]:
# Dataset First Look
df.head()

In [ ]:
df.tail()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
df.shape

### Dataset Information

In [ ]:
# Dataset Info
df.info()

We have Null values in CustomerID and which is importent feature and filling Null values affect our coustmer segementation. So we drop all null values datapoint. which wil give us a dataset of 406 thousand rows.

## ***Understanding Feature  Variables***

#### InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
#### StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
####Description: Product (item) name. Nominal.
#### Quantity: The quantities of each product (item) per transaction. Numeric.
#### InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
#### UnitPrice: Unit price. Numeric, Product price per unit in sterling.
#### CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
#### Country: Country name. Nominal, the name of the country where each customer resides.

### Variables Description

In [ ]:
#DataSet Description
df.describe([0.99,0.90])

#### Missing Values/Null Values

Feature Quantity and UnitPrice have Max value very high in comparision to value at 99 percent which mean there could be outliers.

In [ ]:
# Missing Values/Null Values Count
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

We have null values in CustomerId and we cannt fill these values so we need to drop these values.

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
df.duplicated().value_counts()

In [ ]:
df.drop_duplicates(keep='first',inplace=True)

#### Canceled order

In [ ]:
#Changing datatype to str
df["InvoiceNo"]=df['InvoiceNo'].astype('str')

In [ ]:
#Droping all the Invoice with 'C'
df=df[~df['InvoiceNo'].str.contains('C')]

Droping all the ordered which were canceled . Taking all the Invoice which dont contain 'c' latter

### Checking Distribution of Features.

In [ ]:
#ploting the Distribution of Quantity
plt.figure(figsize=(10,5))
sns.distplot(df['Quantity'],kde=True,color='red',bins=30)
plt.title("Distribution of  Feature  Quantity ")
plt.show()

Data is highly right skewed . We apply some transformation and check the result.

####   Log Transformation of Distribution of Quantity.

In [ ]:
#ploting the Log Transformed Distribution of Quantity
plt.figure(figsize=(10,5))
sns.distplot(np.log1p(df['Quantity']),kde=True,color='red',bins=30)
plt.title("Distribution of  Feature  Quantity ")
plt.show()

This is Much Better then Earlier plot . Less skewed and similer to symmetric plot.

####  Checking the Distribution of UnitPrice

In [ ]:
#ploting the Distribution of UnitPrice
plt.figure(figsize=(10,5))
sns.distplot(df['UnitPrice'],kde=True,color='red',bins=30)
plt.title("Distribution of  Feature  UnitPrice ")
plt.show()

Again the Distribution of UnitPrice is Hightly Right Skewed . We need to Transform the Distribution . We will apply Log Transformation .

#### Log Transformation of UnitPrice Distribution.

In [ ]:
#ploting the Distribution of UnitPrice with Log1P Transformation
plt.figure(figsize=(10,5))
sns.distplot(np.log1p(df['UnitPrice']),kde=True,color='red',bins=30)
plt.title("Distribution of  Feature  UnitPrice ")
plt.show()

Now it is much much better , it is less skewed and more kind of Symmetric

## ***Data Wrangling***

####Creating new Feature From Invoice Date

In [ ]:
#Creating feature from Invoice Date
df['Day_Name']=df.InvoiceDate.dt.day_name()
df['Day_Month']=df.InvoiceDate.dt.day
df['Hour']=df.InvoiceDate.dt.hour
df['Month_Name']=df.InvoiceDate.dt.month_name()
df['Date']=df.InvoiceDate.dt.date

#### Creating Feature 'TotalAmount' From Quantity and UnitPrice

In [ ]:
#Creating New Feature 'TotalAmount' from Quantity and UnitPrice
df['TotalAmount']=df['Quantity']*df['UnitPrice']

### Creating the RFM model (Recency, Frequency,Monetary value)

Recency, frequency, monetary value is a marketing analysis tool used to identify a company's or an organization's best customers by using certain measures. The RFM model is based on three quantitative factors: Recency: When was Last Produch Purchased. Frequency: How often a customer makes a purchase. Monetary Value: How much money a customer spends on

Performing RFM Segmentation and RFM Analysis, Step by Step
The first step in building an RFM model is to assign Recency, Frequency and Monetary values to each customer. ...
The second step is to divide the customer list into tiered groups for each of the three dimensions (R, F and M), using Excel or another tool.

#####Creating the Recency Feature

In [ ]:
#Latest Date (Date of Last Purchase)
Latest_Date=df['Date'].max()
#Getting last purchase Date for each customer
df_r=df.groupby(['CustomerID'],as_index=False)['Date'].max()
#Getting the Recency
df_r['Recency']=(Latest_Date-df_r['Date']).dt.days

##### Creating Frequency Feature

In [ ]:
#Getting Frequency
df_f=df.groupby(['CustomerID'],as_index=False)['Date'].count()
#Renaming Frequecy Coulmn
df_f.rename(columns={'Date':'Frequency'},inplace=True)

##### Creating Monetary Feature

In [ ]:
#Getting Total Amount Spend by each Customer
df_a=df.groupby(['CustomerID'],as_index=False).agg({'TotalAmount':'sum'})
#Renaming the Monetary Function
df_a.rename(columns={'TotalAmount':'Monetary'},inplace=True)

#####Getting Merged Dataset

In [ ]:
#Merging all Data set of Recency Frequency and Monetary
df_m=df_a.merge(df_f,on='CustomerID')
df_rfm=df_m.merge(df_r,on="CustomerID",how='right').drop("Date",axis=1)
df_rfm.head()

#####Calculating RFM scores

The number is typically 5 . If you decide to code each RFM attribute into 3 categories, you'll end up with 125 different coding combinations ranging from a high of 111 to a low of 555. Generally speaking, the Lower the RFM score, the more valuable the customer.

In [ ]:
#Cutting each Feature in Three section according to value and assigning labels
df_rfm['R']=pd.qcut(df_rfm['Recency'],5,labels=[1,2,3,4,5])
df_rfm['F']=pd.qcut(df_rfm['Frequency'],5,labels=[5,4,3,2,1])
df_rfm['M']=pd.qcut(df_rfm['Monetary'],5,labels=[5,4,3,2,1])

In [ ]:
#Creating Grouped 'RMF' Feature
df_rfm['RFM']=df_rfm['R'].astype('str')+df_rfm['F'].astype('str')+df_rfm['M'].astype('str')
df_rfm.head()

## ***EDA And Data Visualization***

#### Highest Selling Product

In [ ]:
#Top five product
df_des=df['Description'].value_counts().sort_values(ascending=False)
df_des.head()

In [ ]:
#ploting the Highest selling product
plt.figure(figsize=(10,5))
df_des.head().plot(kind='bar')
plt.title("Highest selling product")
plt.xlabel("top five product")
plt.ylabel("No of product sold")
plt.show()

***These are top five product according to sale . Highest selling product is WHITE HANGING HEART T-LIGHT HOLDER  and it sold  2016 times. followed by other product.***

####Least Selling Five Product

In [ ]:
#Least selling five product
df_des.tail()

In [ ]:
#ploting the Lowest selling product
plt.figure(figsize=(10,5))
df_des.tail().plot(kind='bar')
plt.title("Lowest selling product")
plt.xlabel("Least selling product")
plt.ylabel("No of product")

***This is least of least selling product . There are many product which are sold single time . I have included only five of them.***

####Country Wise Sales

In [ ]:
#Countries Five Top By Sales
df_cou=df.Country.value_counts()
df_cou.head()

In [ ]:
#Ploting Top five Country by Sales
plt.figure(figsize=(10,5))
df_cou.head().plot(kind='bar')
plt.title("Top five Country By Sales")
plt.xlabel("Countries")
plt.ylabel("Sales Count")

***Top Country according to sales is United Kingdom . Which have nearly 90 percent share . Followed by other Europian Countries like Germany , France and Spain.***

####Last Five Country By Sales


In [ ]:
#Country with Least Sales
df_cou.tail()

In [ ]:
#Ploting Last Five Country By Sales
plt.figure(figsize=(10,5))
df_cou.tail().plot(kind='bar')
plt.title('Last Five Country By Sales')
plt.xlabel('Countries')
plt.ylabel('Sales Count')

***Lithuania,
Brazil ,
Czech Republic,
Bahrain, and
Saudi Arabia are last in list of sales , there could be any reason for it , might be company recently started selling product to these countries .***

####Top Five Stock Counts

In [ ]:
#Top Five Stock usedand there count
df_st=df.StockCode.value_counts()
df_st.head()

In [ ]:
#Ploting Top Five Stock Code
plt.figure(figsize=(10,5))
df_st.head().plot(kind='bar')
plt.title("Top Five Stock Code")
plt.xlabel("Stock Code")
plt.ylabel("Count")
plt.show()

***Each stock represent a particular item or location , these are highest used stock like 85123A is used more than two thousand times.***

####Bottom Five Stock Counts

In [ ]:
#Bottom Five Stock Counts
df_st.tail()

In [ ]:
#Ploting Bottom Five Stock Counts
plt.figure(figsize=(10,5))
df_st.tail().plot(kind='bar')
plt.title("Bottom Five Stock Code")
plt.xlabel("Stock Code")
plt.ylabel("Counts")
plt.show()

#### Orders During Day

In [ ]:
#Plot of orders' time of day
plt.figure(figsize=(10,5))
df.groupby("Hour")["Hour"].count().plot(kind='line')
plt.title('Time of order')
plt.xlabel('Hour of Day')
plt.ylabel("No of orders")
plt.show()

***We can see that most of orders recieved during day time from 8am to 5pm . There are negligible order recieved during morning and evening time***

####Sales During Month

In [ ]:
#Ploting the Sales Count During Month
plt.figure(figsize=(12,5))
df.groupby("Day_Month")['Day_Month'].count().plot(kind='line')
plt.title("Sales Count During Month")
plt.xlabel("Day of Month")
plt.ylabel("Sales Count")
plt.show()

***We can see that count of sales is high at begining of month and low at end of month which might be the reason that at starting of month we get salery and tends to spend but last of month our tendency is to save money.***

#### Purchasing Day Count

In [ ]:
#Ploting the Sales Count During Month
plt.figure(figsize=(12,5))
df.groupby("Day_Name")['Day_Name'].count().sort_values(ascending=False).plot(kind='bar')
plt.title("Sales Count During week")
plt.xlabel("Day of Week")
plt.ylabel("Sales Count")
plt.show()

***We observed More Sales on Thrusday. And least sales is observed on Friday.***

#### Purchasing During Year

In [ ]:
df.groupby("Month_Name")['Month_Name'].count().sort_values(ascending=False)

In [ ]:
#Ploting the Sales Count During Year
plt.figure(figsize=(12,5))
df.groupby("Month_Name")['Month_Name'].count().sort_values(ascending=False).plot(kind='bar')
plt.title("Sales Count During Year")
plt.xlabel("Month of Year")
plt.ylabel("Sales Count")
plt.show()

***Highest Sales were observed in November, October and December Month and lowest were observed in January and February. The reason could be any thing may be because of Season , or Festival Season or some other occassion.***

## ***Hypothesis Testing***

Hypothesis 1: :Do Recent Customer Tends to Spend More.

Hypothesis 2: Do Frequent Customer Spend Large or Small Amount at a Time.


####Creating Parameter Class

In [ ]:
# Creating Parameter Class
class findz:
  def proportion(self,sample,hyp,size):
    return (sample - hyp)/math.sqrt(hyp*(1-hyp)/size)
  def mean(self,hyp,sample,size,std):
    return (sample - hyp)*math.sqrt(size)/std
  def varience(self,hyp,sample,size):
    return (size-1)*sample/hyp

variance = lambda x : sum([(i - np.mean(x))**2 for i in x])/(len(x)-1)
zcdf = lambda x: norm(0,1).cdf(x)

# Creating a function for getting P value
def p_value(z,tailed,t,hypothesis_number,df,col):
  if t!="true":
    z=zcdf(z)
    if tailed=='l':
      return z
    elif tailed == 'r':
      return 1-z
    elif tailed == 'd':
      if z>0.5:
        return 2*(1-z)
      else:
        return 2*z
    else:
      return np.nan
  else:
    z,p_value=stats.ttest_1samp(df[col],hypothesis_number)
    return p_value

  # Conclusion about the P - Value
def conclusion(p):
  significance_level = 0.05
  if p>significance_level:
    return f"Failed to reject the Null Hypothesis for p = {p}."
  else:
    return f"Null Hypothesis rejected Successfully for p = {p}"

# Initializing the class
findz = findz()

### Hypothesis 1. Do Recent Customer Tends to Spend More

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Null Hypothesis: TotalAmount_R <= TotalAmount_avg

Alternate Hypothesis : TotalAmount_R >  TotalAmount_avg

Test Type: right Tailed Test

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value
hypothesis_number=df['TotalAmount'].mean()
df_hypo=df[df['Date']>pd.to_datetime(20111118, format='%Y%m%d')]
sample_mean=df_hypo['TotalAmount'].mean()
size=len(df_hypo)
std=(statistics.variance(df_hypo['TotalAmount']))**0.5

In [ ]:
# Getting Z value
z = findz.mean(hypothesis_number,sample_mean,size,std)
# Getting P - Value
p = p_value(z=z,tailed='r',t="true",hypothesis_number=hypothesis_number,df=df_hypo,col="TotalAmount")
# Getting Conclusion
print(conclusion(p))

##### Which statistical test have you done to obtain P-Value?

I have used T-Test as the statistical testing to obtain P-Value and found the result that we failed to  rejected the Null hypothesis , can say that Recent Transaction doesnt spend Large  Amount in comparisation to average spending per transaction.

##### Why did you choose the specific statistical test?

In [ ]:
mean_median_difference=df_hypo["TotalAmount"].mean()- df_hypo["TotalAmount"].median()
print("Mean Median Difference is :-",mean_median_difference)

From the above chart we can see median is greater than mean over 10. So, the distribution is postively skewed. For a skewed data Z-Test can't be performed.

Non-parametric tests are most useful for small studies. Using non-parametric tests in large studies may provide answers to the wrong question, thus confusing readers. For studies with a large sample size, t-tests and their corresponding confidence intervals can and should be used even for heavily skewed data.

So, for a skewed data we can use T-test for better result. Thus, I used t - test.

###Hypothesis 2. Do Frequent Customer  Spend Large or Small Amount at a Time.

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Null Hypothesis: TotalAmount_invoice_freq == TotalAmount_invoice_avg

Alternate Hypothesis : TotalAmount_invoice_freq != TotalAmount_invoice_avg

Test Type: Two Tailed Test

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value
hypothesis_number=df['TotalAmount'].mean()
df_hypo2=df.merge(df_rfm[df_rfm.F.astype(int)==1]['CustomerID'] , on='CustomerID', how='inner')
sample_mean=df_hypo2['TotalAmount'].mean()
size=len(df_hypo2)
std=(statistics.variance(df_hypo2['TotalAmount']))**0.5

In [ ]:
# Getting Z value
z = findz.mean(hypothesis_number,sample_mean,size,std)
# Getting P - Value
p = p_value(z=z,tailed='d',t="true",hypothesis_number=hypothesis_number,df=df_hypo2,col="TotalAmount")
# Getting Conclusion
print(conclusion(p))

##### Which statistical test have you done to obtain P-Value?

I have used T-Test as the statistical testing to obtain P-Value and found the result that Null hypothesis has been rejected successfully in favour  of Alternative hypothesis and Frequent Customer spend Large or Small Amount at a time.

##### Why did you choose the specific statistical test?

In [ ]:
mean_median_difference=df_hypo2["TotalAmount"].mean()- df_hypo2["TotalAmount"].median()
print("Mean Median Difference is :-",mean_median_difference)

From the above chart we can see median is greater than mean over 10. So, the distribution is postively skewed. For a skewed data Z-Test can't be performed.

Non-parametric tests are most useful for small studies. Using non-parametric tests in large studies may provide answers to the wrong question, thus confusing readers. For studies with a large sample size, t-tests and their corresponding confidence intervals can and should be used even for heavily skewed data.

So, for a skewed data we can use T-test for better result. Thus, I used t - test.

## ***Feature Engineering & Data Pre-processing***

#### Droping the Outliers

In [ ]:
#Getting all the Continuous Variable from the Dataset
cont_var=['Quantity','UnitPrice']

In [ ]:
#Ploting the distribution and Boxplot
for var in cont_var:
    plt.figure(figsize=(12,5))
    plt.subplot(1, 2, 1)
    fig = sns.boxplot(y=df[var])
    fig.set_title('Box Plot of %s'%var)
    plt.subplot(1, 2, 2)
    fig = sns.distplot(df[var])
    plt.title("Distribution Plot of %s"%var)
    plt.show()

Data is Highly Right Skewed and there are outliers in it. we use IsolationForest tree based algorithm for anomaly detection .

In [ ]:
#Using IsolationForest For Predition of Outliers of Anomaly in Data
clf = IsolationForest(n_estimators=100, contamination=0.01, random_state=0)
clf.fit(df[cont_var])

# predict raw anomaly score
df['multivariate_anomaly_score'] = clf.decision_function(df[cont_var])

# prediction of a datapoint category outlier or inlier
df['multivariate_outlier'] = clf.predict(df[cont_var])

In [ ]:
# prediction of a datapoint category outlier or inlier
df['multivariate_outlier'].value_counts()

We predicted anomalies using Isolation Forest . Anomalies are 3905 in number and we drop these anomalies .

In [ ]:
#We found 3905 outliear which we are going to take care
df=df[df['multivariate_outlier']==1]

In [ ]:
df.drop(columns=["multivariate_outlier","multivariate_anomaly_score"],axis=1,inplace=True)

###  Data Transformation

#####Monetary Feature

In [ ]:
#Ploting the distribution of Monetary
plt.figure(figsize=(10,5))
sns.distplot(df_rfm['Monetary'],kde=True,bins=50)
plt.title("Distribution of Monetary")

We can see that data is highly Right Skewed. We apply Log1p Transformation .

In [ ]:
#Ploting the distribution of Monetary
plt.figure(figsize=(10,5))
sns.distplot(np.log1p(df_rfm['Monetary']),kde=True,bins=50)
plt.title("Distribution of Monetary")

Graph looks much better Now so we apply Log Transformation to Monetary Feature.

In [ ]:
#Appling Log Transformation
df_rfm['Monetary']=np.log1p(df_rfm['Monetary'])

####Recency Feature

In [ ]:
#Ploting the distribution of Recency
plt.figure(figsize=(10,5))
sns.distplot(df_rfm['Recency'],kde=True,bins=50)
plt.title("Distribution of Recency")

Graph is Right Skewed and We need to apply Transformation to make it symmetric

In [ ]:
#Ploting the distribution of Log Transformation of Recency
plt.figure(figsize=(10,5))
sns.distplot(np.log1p(df_rfm['Recency']),kde=True,bins=50)
plt.title("Distribution of Recency")

This Looks Much better then Before so we apply Log Transformation to it.

In [ ]:
#Appling Log Transformation
df_rfm['Recency']=np.log1p(df_rfm['Recency'])

#### Frequency Feature

In [ ]:
#Ploting the distribution Frequency
plt.figure(figsize=(10,5))
sns.distplot((df_rfm['Frequency']),kde=True,bins=50)
plt.title("Distribution of Frequency")

Plot is again Highly Right Skewed. We apply log Transformation to it.

In [ ]:
#Ploting the distribution of Log Transformation of Frequency
plt.figure(figsize=(10,5))
sns.distplot(np.log1p(df_rfm['Frequency']),kde=True,bins=50)
plt.title("Distribution of Frequency")

We got some Improvemnt in Distribution of Graph so we apply Log Transformation to it.

In [ ]:
df_rfm['Frequency']=np.log1p(df_rfm['Frequency'])

### 6. Data Scaling

In [ ]:
#Creating Feature list Which we want to Scale
rfm_var=['Recency','Frequency','Monetary']

In [ ]:
#Using Standard Scaler to create instance
scaler=StandardScaler()
#Fitting and Transforming the DATA
df_rfm[rfm_var]=scaler.fit_transform(df_rfm[rfm_var])

***We have applied StanderdScalar to scale the Recency,Frequency and Monetary.***

## ***CUSTOMER SEGMENTATION MODEL IMPLIMENTATION***

### K-MEANS Clustering

####Silhouette Score to get optimum no of clusters.

In [ ]:
#Getting Feature Variable
x=df_rfm[rfm_var].values
#Getting Silhouette Score for clusters
for cluster in range(2,15):
  clu=KMeans(n_clusters=cluster)
  #using K-Means for predition
  preds=clu.fit_predict(x)
  #Scoring with silhouette
  score=silhouette_score(x,preds)
  # Create a subplot with 1 row and 2 columns
  fig, (ax1, ax2) = plt.subplots(1, 2)
  fig.set_size_inches(18, 7)
  ax1.set_xlim([-0.1, 1])
  ax1.set_ylim([0, len(x) + (cluster + 1) * 10])
  sample_silhouette_values=silhouette_samples(x,preds)
  y_lower = 10
  for i in range(cluster):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[preds == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / cluster)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10
  ax1.set_title("The silhouette plot for the various clusters.")
  ax1.set_xlabel("The silhouette coefficient values")
  ax1.set_ylabel("Cluster label")
  # The vertical line for average silhouette score of all the values
  ax1.axvline(x=score, color="red", linestyle="--")
  ax1.set_yticks([])  # Clear the yaxis labels / ticks
  ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
  # 2nd Plot showing the actual clusters formed
  colors = cm.nipy_spectral(preds.astype(float) /cluster)
  ax2.scatter(x[:, 0], x[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')
  # Labeling the clusters
  centers = clu.cluster_centers_
  # Draw white circles at cluster centers
  ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')
  for i, c in enumerate(centers):
      ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')
      ax2.set_title("The visualization of the clustered data.")
      ax2.set_xlabel("Feature space for the 1st feature")
      ax2.set_ylabel("Feature space for the 2nd feature")
      plt.suptitle(f"Silhouette analysis for KMeans clustering with n_clusters = {cluster} with score {score}",fontsize=14, fontweight='bold')


plt.show()

***Silhouette score is used to evaluate the quality of clusters created using clustering algorithms such as K-Means in terms of how well samples are clustered with other samples that are similar to each other. The Silhouette score is calculated for each sample of different clusters. We got highest selhouette score .39 for no_of_clusters =2 , after that score is decreasing. So we can take two as cluster no and fit our data on it.***

####Appling Elbow Method to get optimum no of clusters

In [ ]:
#Storing the cluster no. and  square of distance
sum_of_squares={}
for cluster in range(2,15):
  #K-Means for calcuating the SQUARE Distance
  clu=KMeans(n_clusters=cluster,init='k-means++',max_iter=1000)
  kl=clu.fit(x)
  #Adding to Dictionary file
  sum_of_squares[cluster]=kl.inertia_

#Ploting the ELBOW GRAPH
plt.figure(figsize=(10,5))
sns.pointplot(x=list(sum_of_squares.keys()),y=list(sum_of_squares.values()))
plt.title("Elbow Method for Optimum Clusters")
plt.xlabel("No. of Clusters")
plt.ylabel("Sum of Square Distance")
plt.show()

***Optimum No of Clusters is 3 to 5 using elbow method. we will try and fit data on these.***

####***Predition of Clusters***

In [ ]:
#Getting The Clusters Predition with n=2
clu=KMeans(n_clusters=2)
cl=clu.fit_predict(x)
cent=clu.cluster_centers_

K-Means Clustering is an Unsupervised Learning algorithm, which groups the unlabeled dataset into different clusters. Here K defines the number of pre-defined clusters that need to be created in the process, as if K=2, there will be two clusters, and for K=3, there will be three clusters, and so on.It allows us to cluster the data into different groups and a convenient way to discover the categories of groups in the unlabeled dataset on its own without the need for any training.It is a centroid-based algorithm, where each cluster is associated with a centroid. The main aim of this algorithm is to minimize the sum of distances between the data point and their corresponding clusters.

The algorithm takes the unlabeled dataset as input, divides the dataset into k-number of clusters, and repeats the process until it does not find the best clusters. The value of k should be predetermined in this algorithm.

####3D Visualization of Clustering.

In [ ]:
#Ploting the 3D Graph of clustering
fig=plt.figure(figsize=(15,10))
plt.title("3D Visual of Clustering")
ax=fig.add_subplot(111,projection='3d')
xs=df_rfm['Recency']
ys=df_rfm['Frequency']
zs=df_rfm['Monetary']
ax.scatter(xs,ys,zs,c=cl,s=5,cmap='copper')
plt.scatter(cent[:,0],cent[:,1],c='red',s=300,alpha=.9)
ax.set_xlabel("Recency")
ax.set_ylabel("Frequency")
ax.set_zlabel("Monetary")
plt.show()

### DBSCAN Clustering

In [ ]:
# Fitting Density Based Clustering Technique
dbs=DBSCAN(eps=.5,min_samples=15)
y_db=dbs.fit_predict(x)

In [ ]:
#Ploting the 3D Graph of clustering
fig=plt.figure(figsize=(15,10))
plt.title("3D Visual of Clustering")
ax=fig.add_subplot(111,projection='3d')
xs=df_rfm['Recency']
ys=df_rfm['Frequency']
zs=df_rfm['Monetary']
ax.scatter(xs,ys,zs,c=y_db,s=5,cmap='copper')
ax.set_xlabel("Recency")
ax.set_ylabel("Frequency")
ax.set_zlabel("Monetary")
plt.show()

Density-Based Clustering refers to unsupervised learning methods that identify distinctive groups/clusters in the data, based on the idea that a cluster in data space is a contiguous region of high point density, separated from other such clusters by contiguous regions of low point density.

Density-Based Spatial Clustering of Applications with Noise (DBSCAN) is a base algorithm for density-based clustering. It can discover clusters of different shapes and sizes from a large amount of data, which is containing noise and outliers.

### HIERARCHICAL Clustering

#### Using Dendogram to find Optimum No of Clusters

In this algorithm, we develop the hierarchy of clusters in the form of a tree, and this tree-shaped structure is known as the dendrogram.

In [ ]:
#ploting the Dendogram
plt.figure(figsize=(15,8))
dendrogram=sch.dendrogram(sch.linkage(x, method='ward'))
plt.title("Dendogram graph")
plt.xlabel("Customer")
plt.ylabel("Eculidean distance")
plt.show()

***By Finding largest Vertical Distance without crossing horizontal line. Optimum no of clusters is no of vertical line being crossed by drawing a horigontal line at 90. Which gives 2.***

#### Fitting Agglomerative Clustering

In [ ]:
#Using agglomerative cluster
agl=AgglomerativeClustering(n_clusters=2,affinity='euclidean',linkage='ward')
y_hc=agl.fit_predict(x)

***Agglomerative is a bottom-up approach, in which the algorithm starts with taking all data points as single clusters and merging them until one cluster is left.***

#### Ploting the Clustering on 3d map

In [ ]:
#Ploting the 3D Graph of clustering
fig=plt.figure(figsize=(15,10))
plt.title("3D Visual of Clustering")
ax=fig.add_subplot(111,projection='3d')
xs=df_rfm['Recency']
ys=df_rfm['Frequency']
zs=df_rfm['Monetary']
ax.scatter(xs,ys,zs,c=y_hc,s=5,cmap='copper')
ax.set_xlabel("Recency")
ax.set_ylabel("Frequency")
ax.set_zlabel("Monetary")
plt.show()

***As we have trained our model successfully, now we can visualize the clusters corresponding to the dataset.Here we will use the same lines of code as we did in k-means clustering, except one change. Here we will not plot the centroid that we did in k-means, because here we have used dendrogram to determine the optimal number of clusters.***

# **Conclusion**

In [ ]:

from prettytable import PrettyTable

# Specify the Column Names while initializing the Table
myTable = PrettyTable(['SL No.',"Model_Name",'Data', "Optimal_Number_of_cluster"])

# Add rows
myTable.add_row(['1',"K-Means with silhouette_score ", "RFM", "2"])
myTable.add_row(['2',"K-Means with Elbow methos  ", "RFM", "3"])
myTable.add_row(['3',"Hierarchical clustering  ", "RFM", "2"])
myTable.add_row(['4',"DBSCAN ", "RFM", "2"])
print(myTable)

We have gone through all major clustering technique . With K-Means we predicted for range of 2 to 15 and calcualted the silhouette score and find out that cluster_no 2 is most scored. Next we ploted Elbow graph which was giving cluster_no 3 as optimum no of clusters. We finished taking cluster_no 2 as optimum and predicted the clusters and ploted 3d graph .
Next we have choosen Density based clustering technique. DBSCAN dont require value of clusters before fitting . We predicted the clusters with dbscan and found optimum no of clusters as two.
And at the end I have tried Hierarchical clustering technique. initially ploted dendogram , which is kind of tree type structure to show distance , we concluded that cluster_no 2 as optimum no of clusters by looking vertical distance . We fitted data on Aggloromative which is bottom up approach and where we started with single data point clusters and reached at end with two clusters as optimum no.

With this I conclude , Thanks for reading

 SUSHIL.